<a href="https://colab.research.google.com/github/schwarzRabe/Kama_Noja_Bumaja/blob/main/Hausaufgabe_von_PYTHON_3_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q -U kaggle_environments

In [8]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [9]:

wurf_map = {0:1,1:2,2:0} # словарь "победных" ходов. Ключ - ход, значение - побеждающий код
#0 - rock
#1 - paper
#2 - scissors
last_react_action = None # запоминаем свой последний ход

# 1 ----------- Всегдa бумага----------------------------------------
def rock(observation, configuration):
    wurf = 0
    return wurf

# 2 ----------- Всегдвa камень----------------------------------------
def paper(observation, configuration):
    return 1

# 3 ----------- Всегдa ножницы----------------------------------------
def scissors(observation, configuration):
    return 2

# 4 ----------- Случайный бросок----------------------------------------
def random_wurf(observation, configuration):
    wurf = random.randrange(0, configuration.signs)
    return wurf

# 5 ----------- Повторяет ход соперника---------------------------------
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

# 6 -----------побеждает предыдущий ход соперника----------------------
def reaction(observation, configuration):
    if observation.step > 0:
        return wurf_map.get(observation.lastOpponentAction)
    else:
        return random.randrange(0, configuration.signs)

# 7 -----------побеждает свой предыдущий ход---------------------------

def my_last_react(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    else:
        last_react_action = wurf_map.get(last_react_action)

    return last_react_action
# 8 ----------Крещендо - бумага, ножницы, камень---------------------
def creshendo(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action  = 1
    else:
        last_react_action +=1
        if last_react_action % 3 == 0:
            last_react_action = 0
    return last_react_action

# 9 ----------Cендвич - бумага, ножницы, бумага, ножницы-------------
def sandwich(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action  = 1
    else:
        if observation.step % 2 != 0:
            last_react_action = 2
        else:
            last_react_action = 1

    return last_react_action

# 10 ----Если проиграл, выбирает ход, который побеждает ход соперника.
# если выиграл - cлучайный ход--------------------------------------
def lose_learn(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action  = 1
    last_result = observation.reward
    if last_result < 0:
        last_react_action = wurf_map.get(observation.lastOpponentAction)
    else:
        last_react_action = random.randrange(0, configuration.signs)
    return last_react_action

# 11 ----Если выиграл, выбирает ход, который побеждает ход соперника.
# если проиграл - cлучайный ход--------------------------------------
def win_learn(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action  = 1
    last_result = observation.reward
    if last_result == 1:
        last_react_action = wurf_map.get(observation.lastOpponentAction)
    else:
        last_react_action = random.randrange(0, configuration.signs)
    return last_react_action

# 12 -- случайный выбр со смещением в сторону бумаги = 1--------------
# (т.к. по статистике игроки чаще всего выбирают камень = 0,реже всего ножницы = 2)
def random_paper(observation, configuration):
    return random.choices([0,1,2], weights=[0.25, 0.5, 0.25])[0]

# 13 ---- выбирает случайно, но не повторяет свой ход---------------------
def nicht_repeat(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    else:
        last_react_action = random.choice([x for x in [0,1,2] if x != last_react_action])
    return last_react_action


# словарь стратегий
agents = {
    "rock": rock,
    "paper": paper,
    "scissors": scissors,
    "random_wurf":random_wurf,
    "copy_opponent": copy_opponent,
    "reaction": reaction,
    "my_last_react":my_last_react,
    "creshendo":creshendo,
    "sandwich":sandwich,
    "lose_learn":lose_learn,
    "win_learn":win_learn,
    "random_paper":random_paper,
    "nicht_repeat":nicht_repeat
}
# Создаём окружение для игры "Камень, ножницы, бумага"
env = make("rps")
# выбрали стратегию для тестирования
name_agent = 'lose_learn'
name_agent_untersuchen = agents[name_agent]
# удаляем игрока из списка его противников
del agents[name_agent]

# Тестируем двух агентов:
dict_result = {}  # словарь с результатами для каждого противника

for name, agent in agents.items():
# Тестируем нашего выбранного агента со всеми остальными
    agent_untersuchen_win = 0 # сколько раз выиграл проверяемый агент
    agent_controlle_win = 0 # сколько раз выиграл его противник
    mark_win = 0 # итого результат раунда (1 - выиграл, -1 - проиграл, 0- ничья)
    results = evaluate(
        "rps",
        [name_agent_untersuchen, agent],
        num_episodes=10)
    print(f"Результаты: {results}")
    for result in results:
        if result[0] > result[1]:
            agent_untersuchen_win += 1
        elif result[0] < result[1]:
            agent_controlle_win +=1
    if agent_untersuchen_win > agent_controlle_win:
        mark_win = 1
    elif agent_untersuchen_win < agent_controlle_win:
        mark_win=-1
    print(f'противник {name} ')
#    if mark_win != 0:
#        print(f'Проверяемый агент  победил в {agent_untersuchen_win} раундах')
#        print(f'Его противник {name} победил в {agent_controlle_win} раундах')
#    else:
#        print('Победила дружба!')
    dict_result[name] = mark_win
print(f'Стратегия {name_agent}:')
print(dict_result)
# Интерпритация результата:

# На выходе получаем для игрока стоварь с ключами - названиями стратегий противников и значениямиЖ
# 1 - тестируемый бот выиграл
# -1 - тестируемый бот проиграл
# 0 - ничья

num_list = list(dict_result.values()) # получаем список из значений словаря
# считаем, победі, проигрыши и ничьи

print(f'выиграла {num_list.count(1)}')
print(f'проиграла {num_list.count(-1)}')
print(f'сыграла вничью {num_list.count(0)}')



Результаты: [[20.0, -20.0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [35.0, -35.0], [71.0, -71.0], [30.0, -30.0], [0, 0]]
противник rock 
Результаты: [[71.0, -71.0], [0, 0], [0, 0], [0, 0], [27.0, -27.0], [0, 0], [37.0, -37.0], [0, 0], [42.0, -42.0], [39.0, -39.0]]
противник paper 
Результаты: [[50.0, -50.0], [0, 0], [31.0, -31.0], [31.0, -31.0], [0, 0], [0, 0], [22.0, -22.0], [0, 0], [0, 0], [0, 0]]
противник scissors 
Результаты: [[27.0, -27.0], [0, 0], [0, 0], [0, 0], [-31.0, 31.0], [0, 0], [0, 0], [-29.0, 29.0], [26.0, -26.0], [-31.0, 31.0]]
противник random_wurf 
Результаты: [[-982.0, 982.0], [-999.0, 999.0], [-999.0, 999.0], [-999.0, 999.0], [-999.0, 999.0], [-985.0, 985.0], [-999.0, 999.0], [-999.0, 999.0], [-998.0, 998.0], [-995.0, 995.0]]
противник copy_opponent 
Результаты: [[28.0, -28.0], [63.0, -63.0], [0, 0], [25.0, -25.0], [0, 0], [28.0, -28.0], [0, 0], [0, 0], [0, 0], [54.0, -54.0]]
противник reaction 
Результаты: [[-998.0, 998.0], [-999.0, 999.0], [-997.0, 997.0], [-999.